In [1]:
import pandas
import numpy
import matplotlib.pyplot as plt
from msppy.msp import MSLP
from msppy.solver import Extensive
from msppy.solver import SDDP
from msppy.evaluation import Evaluation
import gurobipy

In [2]:
hydro_ = pandas.read_csv("./data/hydro.csv", index_col=0)
demand = pandas.read_csv("./data/demand.csv", index_col=0)
deficit_ = pandas.read_csv("./data/deficit.csv", index_col=0)
exchange_ub = pandas.read_csv("./data/exchange.csv", index_col=0)
exchange_cost = pandas.read_csv("./data/exchange_cost.csv", index_col=0)
thermal_ = [pandas.read_csv("./data/thermal_{}.csv".format(i), index_col=0) for i in range(4)]
hist = [pandas.read_csv("./data/hist_{}.csv".format(i), sep=";") for i in range(4)]
hist = pandas.concat(hist, axis=1)
hist.dropna(inplace=True)
hist.drop(columns='YEAR', inplace=True)
scenarios = [hist.iloc[:,12*i:12*(i+1)].transpose().values for i in range(4)]

In [3]:
T = 3
HydroThermal = MSLP(T=T, bound=0, discount=0.9906)
for t in range(T):
    m = HydroThermal[t]
    stored_now, stored_past = m.addStateVars(4, ub=hydro_['UB'][:4], name="stored")
    spill = m.addVars(4, name="spill",obj=0.001)
    hydro = m.addVars(4, ub=hydro_['UB'][-4:], name="hydro")    
    deficit = m.addVars(
        [(i,j) for i in range(4) for j in range(4)], 
        ub = [demand.iloc[t%12][i] * deficit_['DEPTH'][j] for i in range(4) for j in range(4)],
        obj = [deficit_['OBJ'][j] for i in range(4) for j in range(4)], 
        name = "deficit")
    thermal = [None] * 4
    for i in range(4):
        thermal[i] = m.addVars(
            len(thermal_[i]), 
            ub=thermal_[i]['UB'], 
            lb=thermal_[i]['LB'], 
            obj=thermal_[i]['OBJ'], 
            name="thermal_{}".format(i)
        )
    exchange = m.addVars(5,5, obj=exchange_cost.values.flatten(),
        ub=exchange_ub.values.flatten(), name="exchange")        
    thermal_sum = m.addVars(4, name="thermal_sum")
    m.addConstrs(thermal_sum[i] == gurobipy.quicksum(thermal[i].values()) for i in range(4))
    
    for i in range(4): 
        m.addConstr(
            thermal_sum[i] 
            + gurobipy.quicksum(deficit[(i,j)] for j in range(4)) 
            + hydro[i] 
            - gurobipy.quicksum(exchange[(i,j)] for j in range(5))
            + gurobipy.quicksum(exchange[(j,i)] for j in range(5)) 
            == demand.iloc[t%12][i]
        )
    m.addConstr(
        gurobipy.quicksum(exchange[(j,4)] for j in range(5)) 
        - gurobipy.quicksum(exchange[(4,j)] for j in range(5)) 
        == 0
    )
    for i in range(4):
        if t == 0:
            m.addConstr(
                stored_now[i] + spill[i] + hydro[i] - stored_past[i] 
                == hydro_['INITIAL'][4:8][i]
            )
        else:
            m.addConstr(
                stored_now[i] + spill[i] + hydro[i] - stored_past[i] == 0, 
                uncertainty = {'rhs': scenarios[i][(t-1)%12]}
            )
    if t == 0:
        m.addConstrs(stored_past[i] == hydro_['INITIAL'][:4][i] for i in range(4))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [4]:
HydroThermal.set_AVaR(lambda_=0.9, alpha_=0.05)

In [5]:
Extensive(HydroThermal).solve(outputFlag=0)

Academic license - for non-commercial use only


1138228.533788362

In [6]:
HydroThermal_SDDP = SDDP(HydroThermal)
HydroThermal_SDDP.solve(logFile=0, n_processes=3, n_steps=12, max_iterations=40)

------------------------------------------------------------------------------------
                             SDDP Solver, Lingquan Ding                             
------------------------------------------------------------------------------------
   Iteration               Bound            Value 95% CI (3)                    Time
------------------------------------------------------------------------------------
           1       821875.171756      692708.492050, 884207.918004          0.273211
           2      1100168.413462      768825.345291, 837227.561569          0.266383
           3      1134930.900226      814023.238666, 916306.542898          0.331615
           4      1135369.122277      819476.638310, 919546.569767          0.346709
           5      1136583.648284      825246.167283, 867377.508013          0.301079
           6      1137213.272971      817392.917674, 860747.634631          0.307362
           7      1137369.732110      818671.746836, 856383.25959

In [7]:
result = Evaluation(HydroThermal)
result.run(n_simulations=-1)
result.epv, numpy.std(result.pv)

(847479.7870219969, 45457.44760661377)